In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password1"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)
rescue_criteria = {
    "Water Rescue": {
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
     "Mountain or Wilderness Rescue": {
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Disaster or Individual Tracking": {
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    },
    "Reset": {}
}

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'gsImage.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.H1('Brashears CS-340 Dashboard')),
    html.Hr(),
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height='75px')
        ),
        html.Div("Created by Hunter Brashears", style={'textAlign': 'right'})
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),

    html.Hr(),

    # Filter Radio Buttons
    html.Div([
        html.Label("Filter Rescue Type"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        )
    ]),


    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[
                             {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                             ],
                             data=df.to_dict('records'),
                             page_size=10,
                             page_current=0,
                             sort_action='native',
                             filter_action='native',
                            row_selectable='single',
                            page_action='native',
                            selected_columns=[],
                            selected_rows=[0],
                            column_selectable=False,
                            row_deletable=False,
                        ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])

])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    query = rescue_criteria.get(filter_type, {})
    df = pd.DataFrame.from_records(db.read(query))
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    return data, columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    if 'breed' in dff.columns and not dff.empty:
        breed_counts = dff['breed'].value_counts()
        top_breeds = breed_counts[:8]
        other_breeds = breed_counts[8:].sum()
        graph_data = top_breeds.append(pd.Series({'Other': other_breeds}))
        fig = px.pie(graph_data, names=graph_data.index, values=graph_data.values,
                      title='Top Breeds')
        return [dcc.Graph(figure=fig)]
    return []
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)

    # Deafult to first row if none is selected
    if index is None:
        row = 0
    else:
        row = index[0] 

    # Get the selected row
    selected_row = dff.iloc[row]
    lat = selected_row.get("location_lat")
    lon = selected_row.get("location_long")

    if lat and lon:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=(lat, lon),
                   zoom=10,
                   children=[
                       dl.TileLayer(id='base-layer-id'),
                       dl.Marker(position=(lat, lon), 
                       children=[
                        dl.Tooltip(selected_row.get("breed", "No breed listed")),
                        dl.Popup([
                            html.H4("Animal Details"),
                            html.P(f"Name: {selected_row.get('name', 'No name')}"),
                            html.P(f"Breed: {selected_row.get('breed', 'NA')}"),
                            html.P(f"Outcome: {selected_row.get('outcome_type', 'NA')}")
                        ])
                       ])
                   ])
        ]

    else:
        return [html.P("No location data available for this animal.")]




app.run_server(debug=True)


Dash app running on http://127.0.0.1:12134/


/tmp/ipykernel_10050/2873915286.py:169: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_10050/2873915286.py:169: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_10050/2873915286.py:169: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_10050/2873915286.py:169: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_10050/2873915286.py:169: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_10050/2873915286.py:169: FutureWarning:

The series.append method is deprecated and will be removed from 